In [10]:
from python_dict_wrapper import wrap
import sys
sys.path.append('../')



In [44]:
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter

def get_anim(result, norm=False, title=""):
    fig = plt.figure(figsize=(4, 4))
    ax = plt.gca()
    im = plt.imshow([[1]], cmap='gray')
    
    if title:
        plt.suptitle(title)
    plt.axis('off')
    nt = result.shape[2]

    rg = None
    if norm:
        rg = [result.min(), result.max()]

    def drawframe(n):
        if n >= nt:
            n = nt - 1
        else:
            im.set_data(result[:, :, :, n])
        if norm:
            im.set_clim(rg)
        return (im)

    # blit=True re-draws only the parts that have changed.
    anim = animation.FuncAnimation(fig, drawframe, frames=nt*2, interval=500, blit=False)
    plt.close()
    return anim

for features in ['ShallowMonkeyNet_pvc1', 'ShallowMonkeyNet_pvc4', 'V1Net']:
    args = wrap({'features': features,
                 'ckpt_root': '../pretrained'})

    from fmri_models import get_feature_model

    model, _, _ = get_feature_model(args)

    if features == 'V1Net':
        w = model.s1.conv1.weight.cpu().detach().numpy()
    else:
        w = model.conv1.weight.cpu().detach().numpy()
    w = w / abs(w).max()
    ani = get_anim(.5 + .5*w.reshape((4, 4, 3, 5, 7, 7)).transpose((0, 4, 1, 5, 2, 3)).reshape((28, 28, 3, 5)), title=features)
        
    writer = FFMpegWriter(fps=10)
    ani.save(f"figures/filters_{features}.gif", writer=writer)  

In [35]:
model

V1Net(
  (s1): ShallowNet(
    (conv1): SymmetricConv3d()
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): LeakyReLU(negative_slope=0.1, inplace=True)
    (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    (dropout): Dropout3d(p=0, inplace=False)
  )
  (skip_conv): Conv3d(64, 32, kernel_size=[1, 1, 1], stride=[1, 1, 1])
  (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (res0): ResBlock(
    (branch1): Conv3d(64, 32, kernel_size=(1, 1, 1), stride=[1, 1, 1], bias=False)
    (branch1_bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (branch2): BottleneckTransform(
      (a): Conv3d(64, 8, kernel_size=[3, 1, 1], stride=[1, 1, 1], padding=[1, 0, 0], bias=False)
      (a_bn): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  